Electric Propulsion Test Code #1

In [1]:
import matplotlib.pyplot as plt
from math import sqrt
import pandas as pd
import numpy as np

Inputs

In [3]:
mars_dist = 1 # Going from 1 AU @ Earth to 1.524 AU @ Mars
accel = 1
thrust = 1
isp = 1
t_burn = 1
cost_per_kg = 1
EP = 1
eff = 1
distance = 1 #dont know what distance this refers to

Small Functions

In [5]:
# Calculate the time it takes to travel to Mars
# Assumes a you turn around at the halfway point and decelerate 
f_ma_calc = 2*sqrt(mars_dist/accel)
mass_flow_rate = thrust/(isp*9.8066)
mass_propellant = mass_flow_rate*t_burn
fuel_cost= mass_propellant*cost_per_kg
vary_thrust = (EP*2*eff)/(isp*9.8066)
solar_irradiance = (6.95e8**2/distance**2)*6.4e7

def secs_to_days(secs):
    return secs/3600/24

Vary Mass Function

In [7]:
pos = 0
v = 0 #m/s
t = 0
dt = 3600 # time step (seconds)
dturn = 1
m_struct = 1
m_fuel = 1

for dturn in range(45,100): #search over possible values for the d_turn as a percent of the total trip distance
    pos = 0
    v = 0 #m/s
    t = 0
    
    Mtot = m_struct + m_fuel
    dmdt = mass_flow_rate

    while pos < mars_dist:
        #update velocity
        if pos < dturn/100*mars_dist:
            v = v + thrust/Mtot*dt #accelerate
        else:
            v = v - thrust/Mtot*dt #deaccelerate
        if v < 0:
            print("fail--didn't reach Mars with turnaround at", dturn, "percent of journey") #stop if Hermes starts to turn around
            break
        pos = pos + v*dt #update position
        EP = 2000000
        Mtot = Mtot - dmdt*dt #update mass
        t = t+dt #update time
        
    if pos > mars_dist:
        print("success--reached Mars with turnaround at", dturn, "percent of journey")
        break
print("time to reach Mars:", t/24/3600, "days")
print("velocity at mars", v, "m/s")

success--reached Mars with turnaround at 45 percent of journey
time to reach Mars: 0.041666666666666664 days
velocity at mars 1800.0 m/s


Vary Mass One way

In [11]:
pos = 0
v = 0 #m/s
t = 0
dt = 3600 # time step (seconds)

    
Mtot = m_struct + m_fuel
sun_to_craft = 1.5e11 + pos
isp = 2600
eff = 0.63

while pos < mars_dist:
    #update velocity
    EP = (6.95e8**2/sun_to_craft**2)*6.4e7*3500*0.4
    thrust = (EP*2*eff)/(isp*9.8066)
    dmdt = thrust/(isp*9.8066)
    v = v + thrust/Mtot*dt #accelerate
        
    Mtot = Mtot - dmdt*dt #update mass
    pos = pos + v*dt #update position
    sun_to_craft = 1.5e11 + pos/3
    t = t+dt #update time
print("time to reach Mars:", t/24/3600, "days")
print("velocity at mars", v, "m/s")
print("thrust at Mars:", thrust, "N")
print("solar irradiance at Mars:", (6.95e8**2/sun_to_craft**2)*6.4e7, "W/m^2")

time to reach Mars: 0.041666666666666664 days
velocity at mars 171098.55497631896 m/s
thrust at Mars: 95.05475276462164 N
solar irradiance at Mars: 1370.184225967728 W/m^2
